# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/output_data/cities.csv")
weather_data

,Unnamed: 0,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
0,0,Piteå,65.32,21.48,51.80,93,1,3.36,SE,1596595874
1,1,Kosonsoy,41.26,71.55,82.40,37,60,4.70,UZ,1596595874
2,2,Grindavik,63.84,-22.43,48.20,100,90,17.22,IS,1596595447
3,3,Castro,-24.79,-50.01,50.31,93,0,4.59,BR,1596595373
4,4,Lebu,-37.62,-73.65,51.13,89,43,6.33,CL,1596595322
...,...,...,...,...,...,...,...,...,...,...
558,558,Zhangjiakou Shi Xuanhua Qu,40.61,115.04,83.86,42,99,8.77,CN,1596596018
559,559,George Town,5.41,100.34,87.01,74,20,8.05,MY,1596596018
560,560,Iregszemcse,46.69,18.19,59.00,90,100,7.00,HU,1596596018
561,561,Sayaxché,16.52,-90.17,75.96,91,100,1.63,GT,1596596019


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#store lat/long locations in a variable
locations = weather_data[["lat", "lng"]]

#store humidity in a variable
humidity = weather_data["humidity"]

In [4]:
#create heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * Max temp: >=70 / <=80
    * Wind Speed: <10mph
    * Cloudiness: Zero
* Drop any rows that don't have these three conditions.

In [5]:
#Narrow down results:
narrowed_df = weather_data.loc[(weather_data["max_temp"] >= 70) & (weather_data["max_temp"] <= 80) & 
                               (weather_data["wind_speed"] <= 10) & (weather_data["cloudiness"] == 0)].dropna()
narrowed_df

,Unnamed: 0,city,lat,lng,max_temp,humidity,cloudiness,wind_speed,country,date
15,15,Udachny,66.42,112.40,75.20,31,0,4.47,RU,1596595421
59,59,Strezhevoy,60.73,77.59,78.80,61,0,4.47,RU,1596595553
76,76,Novoagansk,61.94,76.66,79.54,51,0,2.80,RU,1596595892
151,151,Robertson,36.52,-86.87,72.00,83,0,6.87,US,1596595910
168,168,Mokrousovo,55.81,66.77,72.09,70,0,6.87,RU,1596595914
178,178,Suez,29.97,32.53,72.79,70,0,5.26,EG,1596595917
237,237,Mahon,39.89,4.27,73.40,73,0,6.93,ES,1596595931
238,238,Cabra,37.47,-4.44,75.99,73,0,1.12,ES,1596595581
252,252,Buraidah,26.33,43.98,78.80,34,0,5.82,SA,1596595525
259,259,Zaysan,47.47,84.87,78.93,34,0,6.51,KZ,1596595459


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Since we are only looking at just the cities we don't need any data that its weather related

hotel_df = narrowed_df.loc[:, ["city", "country","lat", "lng"]]

#add "hotel name" column
hotel_df["hotel_name"] = ""

#show dataframe
hotel_df.head()

,city,country,lat,lng,hotel_name
15,Udachny,RU,66.42,112.40,
59,Strezhevoy,RU,60.73,77.59,
76,Novoagansk,RU,61.94,76.66,
151,Robertson,US,36.52,-86.87,
168,Mokrousovo,RU,55.81,66.77,


In [7]:
#hit the google a places api for each cities coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Use parameters of 5000 meters and hotels
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

In [8]:
hotel_list = []

for index, row in hotel_df.iterrows():
    #add in lat, lng, and city_name to avoid NameErrors
    lat = row["lat"]
    lng = row["lng"]
    city_name = row["city"]
    
    #add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    #ectract results to a variable
    results = response['results'] #test successful, get hotels
    
    #get hotels and save to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "hotel_name"] = results[0]['name']
        
    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
print("----End of search ----")

Retrieving Results for Index 15: Udachny.
Missing field/result... skipping.
------------
Retrieving Results for Index 59: Strezhevoy.
Closest hotel in Strezhevoy is Гостиница "Турист".
------------
Retrieving Results for Index 76: Novoagansk.
Missing field/result... skipping.
------------
Retrieving Results for Index 151: Robertson.
Closest hotel in Robertson is Holiday Inn Express & Suites Nashville North - Springfield.
------------
Retrieving Results for Index 168: Mokrousovo.
Closest hotel in Mokrousovo is Tserkov'.
------------
Retrieving Results for Index 178: Suez.
Closest hotel in Suez is LUSINDA HOTEL.
------------
Retrieving Results for Index 237: Mahon.
Closest hotel in Mahon is Casa Ládico Boutique Hotel.
------------
Retrieving Results for Index 238: Cabra.
Closest hotel in Cabra is Hotel Villa María.
------------
Retrieving Results for Index 252: Buraidah.
Closest hotel in Buraidah is Ewaa Express Hotel - Buraydah.
------------
Retrieving Results for Index 259: Zaysan.
Clo

In [9]:
hotel_df

,city,country,lat,lng,hotel_name
58,Chlórakas,CY,34.80,32.41,Elysium Hotel
140,Yangi Marg`ilon,UZ,40.43,71.72,Hotel Mehmon Saroy
259,Ulmi,RO,44.49,25.78,
286,Cutro,IT,39.04,16.98,
337,Mehrān,IR,33.12,46.16,مجتمع رفاهی نور سپاه
417,Charters Towers,AU,-20.10,146.27,Park Hotel Motel
450,Ushtobe,KZ,45.25,77.98,
457,Jalu,LY,29.03,21.55,JALO HOTEL
472,Waddān,LY,29.16,16.14,
487,Pokhara,NP,28.23,83.98,Himalayan Front Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

#add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))